In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# 1. Synthetic dataset generation
np.random.seed(42)
n = 5000
time = np.arange(n)
seasonal = 10 * np.sin(2 * np.pi * time / 50)
trend = 0.05 * time
noise = np.random.normal(0, 2, n)
data = seasonal + trend + noise
df = pd.DataFrame({"value": data})

# 2. Windowing function
def create_sequences(series, window=30):
    X, y = [], []
    for i in range(len(series) - window):
        X.append(series[i:i+window])
        y.append(series[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(df["value"].values)
X_t = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
y_t = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_t, y_t)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# 3. Transformer model
class TimeSeriesTransformer(nn.Module):
    def __init__(self, d_model=32, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(1, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        return self.fc(x[-1])

model = TimeSeriesTransformer()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop
for epoch in range(5):
    for batch_x, batch_y in loader:
        optimizer.zero_grad()
        output = model(batch_x.transpose(0,1))
        loss = criterion(output.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


C:\Users\SHANMUGANATHAN P\AppData\Local\Temp\ipykernel_17672\2378718451.py:38: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
